## Example 1.1.1 ( Steve Shreve )

### Introduction

This section demonstrates how to price a European call option using a replicating portfolio
in a one-period binomial model. We illustrate :

- Building a replicating portfolio using stock and money market
- Computing the no-arbitrage price of the option
- Illustrating arbitrage opportunities if the price deviates

### Model Setup

We consider a market with:
- One risky asset (stock)
- One risk-free asset (money market)
- A European call option with strike price $K$

We assume:
- Initial stock price: $S_0 = 4$
- Up factor: $u = 2$
- Down factor: $d = 0.5$
- Interest rate: $r = 1$ (i.e., 100%)
- European call option strike: $K = 5$

In [5]:
S0 = 4        # Stock price at time 0
u = 2         # Up factor
d = 0.5       # Down factor
r = 1         # Interest rate (100%)
K = 5         # Strike price

### Stock Price Evolution

We compute:
- $S_1(H) = S_0 \cdot u = 8$
- $S_1(T) = S_0 \cdot d = 2$

In [6]:
S1_H = S0 * u
S1_T = S0 * d

### Solving Replicating Equations

To replicate the option, we solve for $\Delta_0$ (shares of stock) and $X_0$ (initial portfolio value) such that:

\
\begin{aligned}
\Delta_0 \cdot S_1(H) + (1 + r)\left(X_0 - \Delta_0 \cdot S_0\right) &= \max(S_1(H) - K, 0) = 3 \\
\Delta_0 \cdot S_1(T) + (1 + r)\left(X_0 - \Delta_0 \cdot S_0\right) &= \max(S_1(T) - K, 0) = 0
\end{aligned}


This gives a system of two equations in two unknowns.

In [7]:
from sympy import symbols, Eq, solve

delta, X = symbols('delta X')
eq1 = Eq(delta * S1_H + (1 + r) * (X - delta * S0), max(S1_H - K, 0))  # = 3
eq2 = Eq(delta * S1_T + (1 + r) * (X - delta * S0), max(S1_T - K, 0))  # = 0

solution = solve((eq1, eq2), (delta, X))
delta_0 = float(solution[delta])
X0 = float(solution[X])

### Portfolio Valuation

Compute:
- Initial cash: $\text{cash}_0 = X_0 - \Delta_0\cdot S_0$
- Final cash: $\text{cash}_1 = (1 + r) \cdot \text{cash}_0$
- Value at $t=1$:
  - $X_1(H) = \Delta_0 \cdot S_1(H) + \text{cash}_1$
  - $X_1(T) = \Delta_0 \cdot S_1(T) + \text{cash}_1$

In [8]:
cash_0 = X0 - delta_0 * S0
cash_1 = (1 + r) * cash_0

X1_H = delta_0 * S1_H + cash_1
X1_T = delta_0 * S1_T + cash_1

### Option Payoffs

The European call option pays:
- $\max(S_1(H) - K, 0) = 3$
- $\max(S_1(T) - K, 0) = 0$

In [9]:
option_H = max(S1_H - K, 0)
option_T = max(S1_T - K, 0)

### Arbitrage Analysis

If market price ≠ $X_0$, arbitrage is possible:

- Sell at $X_0 + 0.01$: invest extra and replicate
- Buy at $X_0 - 0.01$: short-sell stock, use surplus to profit

In [10]:
sell_price = X0 + 0.01
arbitrage_profit_seller = (1 + r) * (sell_price - X0)

buy_price = X0 - 0.01
short_sale_income = delta_0 * S0
surplus = short_sale_income - buy_price - abs(cash_0)
arbitrage_profit_buyer = (1 + r) * surplus

### Final Output

In [13]:
# Final Output
print(f"Stock price S0 = {S0}, u = {u}, d = {d}, r = {r}")
print(f"Strike price K = {K}\n")
print(f"Possible prices: S1(H) = {S1_H}, S1(T) = {S1_T}\n")

print(" Replicating Portfolio:")
print(f"  delta_0 = {delta_0:.4f} shares")
print(f"  Initial value X0 = {X0:.4f}")
print(f"  Initial cash = {cash_0:.4f}")
print(f"  Final cash = {cash_1:.4f}")
print(f"  Portfolio value at H = {X1_H:.4f}")
print(f"  Portfolio value at T = {X1_T:.4f}\n")

print(" Option Payoff:")
print(f"  Payoff at H = {option_H}")
print(f"  Payoff at T = {option_T}\n")

print(" Arbitrage:")
print(f"  Sell for {sell_price:.2f} → Profit: {arbitrage_profit_seller:.4f}")
print(f"  Buy for {buy_price:.2f} → Profit: {arbitrage_profit_buyer:.4f}")

Stock price S0 = 4, u = 2, d = 0.5, r = 1
Strike price K = 5

Possible prices: S1(H) = 8, S1(T) = 2.0

 Replicating Portfolio:
  delta_0 = 0.5000 shares
  Initial value X0 = 1.5000
  Initial cash = -0.5000
  Final cash = -1.0000
  Portfolio value at H = 3.0000
  Portfolio value at T = 0.0000

 Option Payoff:
  Payoff at H = 3
  Payoff at T = 0

 Arbitrage:
  Sell for 1.51 → Profit: 0.0200
  Buy for 1.49 → Profit: 0.0200


In [14]:
#Replication check
assert abs(X1_H - option_H) < 1e-6
assert abs(X1_T - option_T) < 1e-6
print("\n Replication successful. No-arbitrage price = {:.2f}".format(X0))


 Replication successful. No-arbitrage price = 1.50
